# Exercise: Performance Optimization

Optimize the following function.

In [1]:
function work!(A, B, v)
    @assert size(A) == size(B)
    val = zero(eltype(v))
    for i in 1:N
        val = mod(v[i],256)
        A[i,1:N] = B[i,1:N] * (sin(val) * sin(val) - cos(val) * cos(val))
    end
    return A
end

work! (generic function with 1 method)

The following data is **fixed** and **not supposed to be modified**!

In [2]:
# do not modify this cell!

using Random
Random.seed!(42)

N = 8000
B = rand(N,N)
v = rand(Int, N);

const result = work!(zeros(N,N), B, v);

# do not modify this cell!

You can compare against `A_result` to test your implementation(s):

In [3]:
using Test

@test work!(zeros(N,N), B, v) ≈ result

Test Passed

You can benchmark as follows:

In [4]:
using BenchmarkTools

@btime work!(A, $B, $v) setup=(A=zeros(N,N)); # or use @benchmark for more information

  550.996 ms (150979 allocations: 1002.91 MiB)


## Your Optimizations

Your optimized variants go here!

**Hints** (hopefully):
* What is suboptimal about the code? What is it that you'd want to change (but can't directly)?
* Sometimes writing the code in a different way doesn't give direct speedups but enables further optimization.
* A ~30x speedup should be possible on most systems 😉

### Fixing the type instability (accessing global `N`)

In [5]:
@code_warntype work!(zeros(N,N), B, v)

MethodInstance for work!(::Matrix{Float64}, ::Matrix{Float64}, ::Vector{Int64})
  from work!(A, B, v) @ Main In[1]:1
Arguments
  #self#::Core.Const(Main.work!)
  A::Matrix{Float64}
  B::Matrix{Float64}
  v::Vector{Int64}
Locals
  @_5::Any
  val::Any
  i::Any
Body::Matrix{Float64}
1 ─       Core.NewvarNode(:(@_5))
│         Core.NewvarNode(:(val))
│   %3  = Main.:(==)::Core.Const(==)
│   %4  = Main.size::Core.Const(size)
│   %5  = (%4)(A)::Tuple{Int64, Int64}
│   %6  = Main.size::Core.Const(size)
│   %7  = (%6)(B)::Tuple{Int64, Int64}
│   %8  = (%3)(%5, %7)::Bool
└──       goto #3 if not %8
2 ─       goto #4
3 ─ %11 = Base.throw::Core.Const(throw)
│   %12 = Base.AssertionError("size(A) == size(B)")::Core.Const(AssertionError("size(A) == size(B)"))
└──       (%11)(%12)
4 ┄ %14 = Main.zero::Core.Const(zero)
│   %15 = Main.eltype::Core.Const(eltype)
│   %16 = (%15)(v)::Core.Const(Int64)
│         (val = (%14)(%16))
│   %18 = Main.:(:)::Core.Const(Colon())
│   %19 = Main.N::Any
│   %20 = (%

Get `N` from the size of `A` (or `B`) or add another function argument.

In [7]:
function work1!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1) # or additional function argument
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        A[i,1:N] = B[i,1:N] * val
    end
    return A
end

@btime work1!(A, $B, $v) setup=(A=zeros(N,N))
@test work1!(zeros(N,N), B, v) ≈ result

  517.560 ms (48000 allocations: 1000.98 MiB)


Test Passed

### Analytic optimization (style points 😉)

Trigonometric identity

In [8]:
x = rand()
@test sin(x) * sin(x) - cos(x) * cos(x) ≈ -cos(2*x)

Test Passed

In [9]:
function work2!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        A[i,1:N] = B[i,1:N] * val
    end
    return A
end

@btime work2!(A, $B, $v) setup=(A=zeros(N,N))
@test work2!(zeros(N,N), B, v) ≈ result

  511.549 ms (48000 allocations: 1000.98 MiB)


Test Passed

### Avoid allocations due to slicing (`B[i, 1:N]`)

In [13]:
function work3_vectorized!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        @views A[i,1:N] = B[i,1:N] * val
    end
    return A
end

@btime work3_vectorized!(A, $B, $v) setup=(A=zeros(N,N))
@test work3_vectorized!(zeros(N,N), B, v) ≈ result

  414.931 ms (24000 allocations: 500.49 MiB)


Test Passed

Same idea but explicit loop + `@inbounds`

In [12]:
function work3_loop!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))     
        for j in 1:N
            @inbounds A[i,j] = B[i,j] * val
        end
    end
    return A
end

@btime work3_loop!(A, $B, $v) setup=(A=zeros(N,N))
@test work3_loop!(zeros(N,N), B, v) ≈ result

  475.580 ms (0 allocations: 0 bytes)


Test Passed

### Separating `val` computation

In [14]:
function work4_vectorized!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for i in 1:N
        @views A[i,1:N] .= B[i,1:N] .* val[i]
    end
    return A
end

@btime work4_vectorized!(A, $B, $v) setup=(A=zeros(N,N))
@test work4_vectorized!(zeros(N,N), B, v) ≈ result

  492.305 ms (3 allocations: 64.06 KiB)


Test Passed

Same idea but explicit loop + `@inbounds`

In [15]:
function work4_loop!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for i in 1:N
        for j in 1:N
            @inbounds A[i,j] = B[i,j] * val[i]
        end
    end
    return A
end

@btime work4_loop!(A, $B, $v) setup=(A=zeros(N,N))
@test work4_loop!(zeros(N,N), B, v) ≈ result

  1.094 s (3 allocations: 64.06 KiB)


Test Passed

### Switch loop order (!!!)

In [16]:
function work5!(A, B, v)
    @assert size(A) == size(B)
    N = size(A,1)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for j in 1:N
        for i in 1:N
            @inbounds A[i,j] = B[i,j] * val[i]
        end
    end
    return A
end

@btime work5!(A, $B, $v) setup=(A=zeros(N,N))
@test work5!(zeros(N,N), B, v) ≈ result

  23.563 ms (3 allocations: 64.06 KiB)


Test Passed

## Bonus Question: Performance limit?

Look at your final optimized version of `work!`.

* In the limit of larger `A` and `B`, what is conceptually limiting the performance, the compute capability or memory transfer (i.e. reading and writing `A` and `B`)?

Let's try to quickly estimate the maximal memory bandwidth that a single-CPU core can achieve on the given computer:

In [17]:
using STREAMBenchmark
membw = memory_bandwidth(; nthreads=1).median / 1000 # memory bandwidth in GB /s

90.27210000000001

For references, a single CPU-core in [Noctua 2](https://pc2.uni-paderborn.de/systems-and-services/noctua-2) can achieve a **maximal memory bandwidth of ~45 GB/s**.

* Given the maximal memory bandwidth, can you give a performance bound estimate, i.e. the minimal runtime that we could possibly hope to achieve?
  * Hint: how many flops are performed per iteration and how many bytes are transferred?
* How far off is your implementation from achieving the limit (in percent)?

In [18]:
# membw = 45 # GB/s
flops = 1 # flops per iteration
traffic = 3*8 # bytes per iteration
I = flops / traffic # flops / byte

perf_bound = I*membw # GFLOPS
runtime_estimate = N^2 * 1e3 / (perf_bound * 1e9) # in ms

println("Performance bound: ", round(perf_bound, digits=2), " GFLOP/s")
println("Runtime estimate: ", round(runtime_estimate, digits=2), " ms")

Performance bound: 3.76 GFLOP/s
Runtime estimate: 17.02 ms


In [19]:
t_work5 = @belapsed work5!(A, $B, $v) setup=(A=zeros(N,N))
ratio = runtime_estimate / (t_work5 * 1e3)
println("My best version achieves ", round(ratio * 100, digits=2), "% of the limit.")

My best version achieves 91.6% of the limit.
